# TimeloopFE Processors Overview

TimeloopFE lets users define processors to automate parts of the modeling flow.
This tutorial provides an overview of processors and how they can be used to
introduce new automations.

# The Basic Processor
TimeloopFE processors have two main functions: `declare_attrs` and `process`.
- `declare_attrs` is called once when the processor is initialized. This function
  should declare any attributes that the processor will add to the specification.
  If a processor declares attributes, it should also remove them in the `process`
  function.
- `process` is called once in the Specification's `process` function. The `process`
  function should edit the specification and must remove any attributes that the
  processor has declared.

An example processor is shown below. The processor declares an attribute,
"simple_processor_attr", in the Problem. In the process function, it deletes
this attribute.

In [1]:
import timeloopfe.v4 as tl
import os
TOP_PATH = f"{os.curdir}/top.yaml.jinja"

class MyProcessor(tl.Processor):
    """!@brief An example simple processor."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.logger.info("Initializing SimpleProcessor")

    def init_elems(self):
        """!@brief Initialize the attributes that the processor handles."""
        super().init_elem(tl.Problem, "simple_processor_attr", str, "")

    def process(self, spec: tl.Specification):
        """!@brief Process the specification. Remove attributes that this
        processor is responsible for."""
        if "simple_processor_attr" in spec.problem:
            del spec.problem["simple_processor_attr"]
            self.logger.info('Deleted "simple_processor_attr"')


## Processor Usage Example
Let's create a proccessor that expands buffers in the architecture. 

In the `declare_attrs` function, this processor defines an `expansion_factor`
attribute that determines by how much it will expand a given buffer. In the
`process` function, it modifies the depth/width of architecture components and
deletes these expansion factors.

In the architecture YAML, we can define `expansion_factor` attributes to be
parsed by this processor.

In [2]:
DELETE_EXPANSION_FACTOR = True

class BufferExpansionProcessor(tl.Processor):
    """!@brief Expands buffers in the architecture."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Each processor
        self.logger.info("Initializing BufferExpansionProcessor")

    def declare_attrs(self):
        """!@brief Initialize the attributes that the processor handles."""
        # Initialize for the Component class, name "expansion_factor", type int,
        # default value 1
        super().add_attr(tl.arch.Component, "expansion_factor", int, 1)

    def process(self, spec: tl.Specification):
        for component in spec.architecture.get_nodes_of_type(tl.arch.Component):
            print(f'Looking at {component}')
            expansion_factor = component.expansion_factor
            if DELETE_EXPANSION_FACTOR:
                component.pop('expansion_factor')

            self.logger.warning(
                f"Expanding buffer {component.name} by {expansion_factor}x"
            )
            if component.attributes.get("width", None):
                component.attributes["width"] *= expansion_factor
                self.logger.info(
                    f"Expanded buffer {component.name} by {expansion_factor}x"
                )
            if component.attributes.get("depth", None):
                component.attributes["depth"] *= expansion_factor
                self.logger.info(
                    f"Expanded buffer {component.name} by {expansion_factor}x"
                )


The `declare_attrs` function adds the `expansion_factor` attribute to the
`Component` class. The `process` function expands each buffer in the
architecture by the expansion factor and then deletes the expansion factor
attribute. The `must_run_after` function ensures that the MathProcessor runs
before the BufferExpansionProcessor so that all math expressions are resolved
before the expansion factor is used.

Let's try running the processor on a simple architecture. We set an expansion
factor of 3 for the DRAM and 5 for the buffer. We can see that the processor
expands both the DRAM and the buffer by the correct amount. Other architecture
components are expanded by 1x because the default expansion factor is 1. Below,
we print the width of the DRAM and buffer before and after processing with the
BufferExpansionProcessor.

If processors are provided, then process() must be called before any invocation
of tl.call_mapper, tl.call_model, or tl.call_accelerg_verbose with the
specification.

In [3]:
spec = tl.Specification.from_yaml_files(
    TOP_PATH, processors=[BufferExpansionProcessor]
)
print(tl.get_property_table(tl.arch.Component))
spec.architecture.find("DRAM").expansion_factor = 3
spec.architecture.find("buffer").expansion_factor = 5
print(f"DRAM width before: ", spec.architecture.find("DRAM").attributes["width"])
print(f"Buffer width before: ", spec.architecture.find("buffer").attributes["width"])
spec.process()
print(f"DRAM width after: ", spec.architecture.find("DRAM").attributes["width"])
print(f"Buffer width after: ", spec.architecture.find("buffer").attributes["width"])



====================================================  timeloopfe.v4.arch  ===================================================

==== Component ====
  KEY                      ,REQUIRED_TYPE            ,DEFAULT                  ,CALLFUNC                 ,SET_FROM                 
  ignore                   ,None                     ,None                     ,None                     ,None                     
  name                     ,str                      ,REQUIRED                 ,None                     ,None                     
  attributes               ,Attributes               ,{}                       ,Attributes               ,None                     
  spatial                  ,Spatial                  ,{}                       ,Spatial                  ,None                     
  constraints              ,ConstraintGroup          ,{}                       ,ConstraintGroup          ,None                     
  sparse_optimizations     ,SparseOptimizationGroup  ,{}   

### Processor Exceptions

Processors can raise exceptions if they encounter invalid inputs. For example,
the `must_run_after` function raises an exception if the processor that it
depends on is not found in the list of processors. Here, we try to run the
BufferExpansionProcessor first, but it depends on the MathProcessor in the
standard suite.


In [4]:
spec = tl.Specification.from_yaml_files(
    TOP_PATH, processors=[BufferExpansionProcessor]
)
try:
    spec.process()
except tl.ProcessorError as e:
    print(e)

Looking at Specification[architecture].Architecture[nodes].ArchNodes[1].Storage(DRAM)
Looking at Specification[architecture].Architecture[nodes].ArchNodes[2].Storage(buffer)
Looking at Specification[architecture].Architecture[nodes].ArchNodes[4].Storage(reg)
Looking at Specification[architecture].Architecture[nodes].ArchNodes[5].Compute(mac)


TimeloopFE also raises an exception if a processor does not remove the attributes
that it declares.

In [5]:
spec = tl.Specification.from_yaml_files(
    TOP_PATH, processors=[BufferExpansionProcessor]
)
try:
    DELETE_EXPANSION_FACTOR = False
    spec.process()
except tl.ProcessorError as e:
    print(e)


Looking at Specification[architecture].Architecture[nodes].ArchNodes[1].Storage(DRAM)
Looking at Specification[architecture].Architecture[nodes].ArchNodes[2].Storage(buffer)
Looking at Specification[architecture].Architecture[nodes].ArchNodes[4].Storage(reg)
Looking at Specification[architecture].Architecture[nodes].ArchNodes[5].Compute(mac)


# Editing The Specification and `process()`
The process() function transforms the specification at many points. Users may also edit the specification, and there is an important choice as to whether to edit specifications before or after calling `process()`. TimeloopFE also supports calling `process()` with a list of processors, allowing users to `process()`, edit the specification, `process()` with a different set of processors, and so on.

## Edit Before Process
Generally, we recommend editing the specification *before* calling `process()`.
This is for two reasons. First, Before calling `process()`, we can look at the
YAML files and use them as a reference to know what we're editing. After
`process()`, we can still dump the specification or use print/log statements
to see the specification, but it is more difficult.

Furthermore, formulas defined in the YAML files are resolved during processing.
Editing the calculated results after they have been processed may result in
unexpected behavior. As an example, the variables are propagated from the
variables file during processing. Below, we edit an architecture where the
datawidth of a buffer is set from a DATAWIDTH variable. If we change the
DATWIDTH variable before processing, the new value is propagated to the rest of
the specification. If we incorrectly change the DATWIDTH variable after
processing, the new value is not propagated.

In [6]:

spec = tl.Specification.from_yaml_files(TOP_PATH)


print(f"Buffer datawidth is set to {spec.architecture.find('buffer').attributes['datawidth']}")

print(f"Setting variables DATAWIDTH to 32 BEFORE processing")
spec.variables["DATAWIDTH"] = 32

print(f'\nCalling process()\n')
spec.process()

print(f"After processing, buffer datawaidth is {spec.architecture.find('buffer').attributes['datawidth']}")
print(f"INCORRECT: Setting variables DATAWIDTH to 64 AFTER processing")
spec.variables["DATAWIDTH"] = 64
print(f"Buffer datawaidth is {spec.architecture.find('buffer').attributes['datawidth']}; NOT UPDATED.")


Buffer datawidth is set to DATAWIDTH
Setting variables DATAWIDTH to 32 BEFORE processing

Calling process()

After processing, buffer datawaidth is DATAWIDTH
INCORRECT: Setting variables DATAWIDTH to 64 AFTER processing
Buffer datawaidth is DATAWIDTH; NOT UPDATED.


## Editing Python Expressions
Python expressions can be used to define variables in the `variables`,
`problem`, architecture `attributes`, among other locations. When editing these
expressions, care should be taken to avoid performing arithmetic directly on the
string.

To edit expressions, you may parentheses-wrapping them then add additional
arithmetic to the beginning or end of the expression. The below example shows a
number and expression both being multiplied by four in Python. When the
expression, which is a string, is multiplied by four directly, the string is
copied four times to produce an incorrect result. Instead, we can get a correct
result by parentheses-wrapping the expression and adding a " * 4" to the end.


In [7]:
spec = tl.Specification.from_yaml_files(TOP_PATH)
variables = spec.variables = spec.variables
variables["number"] = 13
variables["formula"] = "8 + 5"


variables["'number * 4'"] = variables["number"] * 4
variables["'(number) * 4'"] = f'({variables["number"]}) * 4'

# INCORRECT: Multiply the string by 4
variables["INCORRECT 'formula * 4'"] = variables["formula"] * 4
# Correct: Parentheses-wrap the string and add a "*4" to create
# a new string containing"(formula) * 4"
variables["CORRECT '(formula) * 4'"] = f'({variables["formula"]}) * 4'

print(f'Variables before processing: ')
for k, v in variables.items():
    if "number" in k or "formula" in k:
        print(f'  {k:25s}: {v}')

spec.process()

print(f'\nVariables after processing: ')
for k, v in variables.items():
    if "number" in k or "formula" in k:
        print(f'  {k:25s}: {v}')

Variables before processing: 
  number                   : 13
  formula                  : 8 + 5
  'number * 4'             : 52
  '(number) * 4'           : (13) * 4
  INCORRECT 'formula * 4'  : 8 + 58 + 58 + 58 + 5
  CORRECT '(formula) * 4'  : (8 + 5) * 4

Variables after processing: 
  number                   : 13
  formula                  : 8 + 5
  'number * 4'             : 52
  '(number) * 4'           : (13) * 4
  INCORRECT 'formula * 4'  : 8 + 58 + 58 + 58 + 5
  CORRECT '(formula) * 4'  : (8 + 5) * 4
